## Web Scraping

In [1]:
import requests
import re
from bs4 import BeautifulSoup, Comment

import json
import numpy as np
import pandas as pd
import time

from pymongo import MongoClient




1978-1979 season to the 2017-2018 season

# Generate URLs for all the seasons of interest from Basketball reference 

In [9]:
url = 'https://www.basketball-reference.com/leagues/NBA_'

In [10]:
def url_gen(url):
    url_list = []
    for i in range(1979, 2019):
        url_list.append(url + str(i) + '.html')
    return url_list

In [11]:
season_urls = url_gen(url)

In [12]:
season_2017 = season_urls[-1]

In [13]:
season_2017

'https://www.basketball-reference.com/leagues/NBA_2018.html'

In [14]:
webpage = requests.get(season_2017)

Miscellaneous table tags: class="sortable stats_table now_sortable" id="misc_stats"

Team shooting: class="sortable stats_table now_sortable" id="team_shooting"

Team stats: class="sortable stats_table now_sortable is_sorted" id ="team-stats-base"

In [15]:
soup_2017 = BeautifulSoup(webpage.text, 'lxml')


In [16]:
for comment in soup_2017.find_all(string=lambda text:isinstance(text,Comment)):
    data = BeautifulSoup(comment,"lxml")
    for items in data.select("table.stats_table tr"):
        tds = [item.get_text(strip=True) for item in items.select("th,td")]
        print(tds)

['Finals', 'Golden State WarriorsoverCleveland Cavaliers(4-0)', 'Series Stats']
['Game 1Thu, May 31Cleveland Cavaliers114@ Golden State Warriors124Game 2Sun, June 3Cleveland Cavaliers103@ Golden State Warriors122Game 3Wed, June 6Golden State Warriors110@ Cleveland Cavaliers102Game 4Fri, June 8Golden State Warriors108@ Cleveland Cavaliers85', 'Game 1', 'Thu, May 31', 'Cleveland Cavaliers', '114', '@ Golden State Warriors', '124', 'Game 2', 'Sun, June 3', '103', '122', 'Game 3', 'Wed, June 6', 'Golden State Warriors', '110', '@ Cleveland Cavaliers', '102', 'Game 4', 'Fri, June 8', '108', '85']
['Game 1', 'Thu, May 31', 'Cleveland Cavaliers', '114', '@ Golden State Warriors', '124']
['Game 2', 'Sun, June 3', 'Cleveland Cavaliers', '103', '@ Golden State Warriors', '122']
['Game 3', 'Wed, June 6', 'Golden State Warriors', '110', '@ Cleveland Cavaliers', '102']
['Game 4', 'Fri, June 8', 'Golden State Warriors', '108', '@ Cleveland Cavaliers', '85']
['']
['Eastern Conference Finals', 'Clevel

['Rk', 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
['1', 'Golden State Warriors*', '82', '19730', '42.9', '85.3', '.503', '11.3', '28.9', '.391', '31.5', '56.3', '.560', '16.6', '20.4', '.815', '8.4', '35.1', '43.6', '29.3', '8.0', '7.5', '15.5', '19.6', '113.6']
['2', 'Houston Rockets*', '82', '19755', '39.5', '86.0', '.460', '15.6', '43.2', '.362', '23.9', '42.8', '.558', '20.0', '25.7', '.781', '9.2', '35.2', '44.4', '22.0', '8.7', '4.9', '14.1', '19.9', '114.7']
['3', 'New Orleans Pelicans*', '82', '19955', '41.9', '86.6', '.483', '10.0', '27.7', '.362', '31.9', '59.0', '.540', '15.8', '20.5', '.772', '8.5', '35.0', '43.5', '26.3', '7.9', '5.8', '14.6', '18.8', '109.6']
['4', 'Toronto Raptors*', '82', '19830', '42.0', '89.1', '.472', '12.0', '33.6', '.358', '30.0', '55.5', '.541', '17.7', '22.2', '.794', '9.9', '34.9', '44.8', '24.8', '7.8', '6.2', '13.6', '22.2', '1

In [21]:
misc_tables = soup_2017.find_all('div', id="all_misc_stats")[0]

In [ ]:
misc_tables

In [ ]:
misc_tables.find_all(string=lambda text:isinstance(text,Comment))[0]

In [22]:
misc = []
for comment in misc_tables.find_all(string=lambda text:isinstance(text,Comment)):
    data = BeautifulSoup(comment,"lxml")
    for items in data.select("table.stats_table tr"):
        tds = [item.get_text(strip=True) for item in items.select("th,td")]
        misc.append(tds)
miscar = pd.DataFrame(misc[2:], columns= misc[1])       
miscar

,Rk,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA,Arena,Attend.,Attend./G
0,1,Houston Rockets*,29.8,65,17,61,21,8.48,-0.27,8.21,...,12.7,21.3,.233,.521,13.4,79.9,.171,Toyota Center,"732,722","17,871"
1,2,Toronto Raptors*,25.8,59,23,60,22,7.78,-0.49,7.29,...,12.1,23.0,.198,.501,13.0,77.7,.212,Air Canada Centre,"813,431","19,840"
2,3,Golden State Warriors*,28.8,58,24,56,26,5.98,-0.19,5.79,...,14.1,21.0,.195,.504,12.6,76.3,.186,Oracle Arena,"803,436","19,596"
3,4,Utah Jazz*,26.7,48,34,53,29,4.30,0.17,4.47,...,13.7,21.5,.202,.506,14.0,79.8,.190,Vivint Smart Home Arena,"734,806","17,922"
4,5,Philadelphia 76ers*,25.8,52,30,53,29,4.50,-0.20,4.30,...,14.6,25.3,.198,.492,12.6,78.6,.218,Wells Fargo Center,"833,503","20,361"
5,6,Oklahoma City Thunder*,27.3,48,34,50,32,3.41,0.00,3.42,...,12.4,27.7,.197,.525,14.7,77.4,.193,Chesapeake Energy Arena,"746,322","18,203"
6,7,Boston Celtics*,24.7,55,27,51,31,3.59,-0.35,3.23,...,13.0,21.5,.188,.495,13.0,78.4,.191,TD Garden,"763,584","18,624"
7,8,San Antonio Spurs*,29.3,47,35,49,33,2.89,0.00,2.89,...,12.2,23.7,.189,.507,13.2,78.6,.170,AT&T Center,"754,562","18,404"
8,9,Portland Trail Blazers*,25.6,49,33,48,34,2.60,0.00,2.60,...,12.3,23.3,.192,.503,11.7,79.1,.193,Moda Center,"795,750","19,409"
9,10,Minnesota Timberwolves*,27.1,47,35,47,35,2.23,0.12,2.35,...,11.4,24.4,.225,.539,13.9,76.4,.189,Target Center,"699,308","17,056"


In [ ]:
attempt = soup_2017.find_all('div', class_ = "div_misc_stats")

In [ ]:
team_stats_table = soup_2017.find_all('div', id='all_team-stats-base')[0]

In [ ]:
shooting_table = soup_2017.find_all('div', id='all_team_shooting')[0]

In [ ]:
#misc_tables.find_all('table', class_ = "sortable stats_table")

In [ ]:
misc_tables

In [ ]:
misc_tables.div


In [ ]:
miscar = pd.DataFrame(misc[2:], columns= misc[1])

In [ ]:
miscar


In [2]:
client = MongoClient()
database = client['test']   # Database name
mongo_connect = database['season_page'] 

In [3]:
season_urls_test = ["https://www.basketball-reference.com/leagues/NBA_2017.html","https://www.basketball-reference.com/leagues/NBA_2018.html"]

In [ ]:
# fields = {}
# fields['misc_tables'] = str(misc_tables)
# fields['team_stats'] = str(team_stats_table)
# fields['season'] = 1
# mongo_connect.insert_one(fields)

In [18]:
fields

{'misc_tables': '<div class="table_wrapper setup_commented commented" id="all_misc_stats">\n<div class="section_heading">\n<span class="section_anchor" data-label="Miscellaneous Stats" id="misc_stats_link"></span><h2>Miscellaneous Stats</h2> <div class="section_heading_text">\n<ul>\n</ul>\n</div>\n</div>\n<div class="placeholder"></div>\n<!--\n   <div class="table_outer_container">\n      <div class="overthrow table_container" id="div_misc_stats">\n  <table class="sortable stats_table" id="misc_stats" data-cols-to-freeze=2><caption>Miscellaneous Stats Table</caption>\n   <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup>\n   <thead>\n      \n      <tr class="over_header">\n         <th aria-label="" data-stat="" colspan="17" class=" over_header center" ></th>\n         <th aria-label="" data-stat="header_team" colspan="4" class="bold_text over_header center" >Offense Four Fact

In [28]:
season = 2000
for i in season_urls_test:
    webpage = requests.get(i)
    fields = {}
    
    soup = BeautifulSoup(webpage.text, 'lxml')
    fields['misc_tables'] = str(soup.find_all('div', id="all_misc_stats")[0])
    fields['team_stats_table'] = str(soup.find_all('div', id='all_team-stats-base')[0])
    fields['shooting_table'] = str(soup.find_all('div', id='all_team_shooting')[0])
    fields['season'] = season
    season += 1
#     with open('scrape_records_test.log', 'a+') as log:
#         log.write(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()))
#         log.write('Webpage: {},'.format(webpage))
    mongo_connect.insert_one(fields)
    time.sleep(5)



In [ ]:
# BeautifulSoup(str(misc_tables), 'lxml')

NameError: name 'scrape_recoords_test' is not defined